# **Projek Klasifikasi Simpel**

In [ ]:
!pip install langchain_community
!pip install replicate
!pip install pandas
!pip install -U langchain langchain-community langchain-experimental replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 26.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.70
    Uninstalling langchain-core-0.3.70:
      Successfully uninstalled langchain-core-0.3.70
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.8
    Uninstalling langchain-text-splitters-0.3.8:
      Successfully uninstalled langchain-text-splitters-0.3.8
  Attempting uninstall: langchain
    F

In [ ]:
import pandas as pd
df = pd.read_csv("/content/favorite_music_dataset.csv")
df

,Song_Title,Artist,Genre,Release_Year,Duration_Minutes,Listened_Date,Platform
0,Faded,Alan Walker,Electronic,2024,4.02,2024-02-10,Apple Music
1,Blinding Lights,The Weeknd,Synth-pop,2018,4.45,2024-05-30,Zing MP3
2,Faded,Alan Walker,Electronic,2024,4.86,2024-05-07,Zing MP3
3,Faded,Alan Walker,Electronic,2012,4.92,2024-03-19,YouTube
4,Bohemian Rhapsody,Queen,Rock,2023,3.90,2024-01-16,Apple Music
...,...,...,...,...,...,...,...
95,Counting Stars,OneRepublic,Pop Rock,2013,3.40,2024-05-30,Spotify
96,Senorita,Shawn Mendes & Camila Cabello,Latin Pop,2024,2.75,2024-06-11,Apple Music
97,Counting Stars,OneRepublic,Pop Rock,2010,3.57,2024-01-03,Zing MP3
98,Blinding Lights,The Weeknd,Synth-pop,2012,4.04,2024-06-13,Apple Music


In [ ]:
df_cleaned = df.drop(columns=["Genre", "Duration_Minutes", "Listened_Date"])

In [ ]:
df_sorted = df_cleaned.sort_values(by="Release_Year", ascending=True).reset_index(drop=True)

# Menampilkan hasil
print(df_sorted)

           Song_Title                         Artist  Release_Year  \
0             Perfect                     Ed Sheeran          2010   
1            Senorita  Shawn Mendes & Camila Cabello          2010   
2    Someone Like You                          Adele          2010   
3        Dance Monkey                    Tones and I          2010   
4                Stay  The Kid LAROI & Justin Bieber          2010   
..                ...                            ...           ...   
95  Bohemian Rhapsody                          Queen          2024   
96   Someone Like You                          Adele          2024   
97    Blinding Lights                     The Weeknd          2024   
98             Lovely         Billie Eilish & Khalid          2024   
99           Senorita  Shawn Mendes & Camila Cabello          2024   

       Platform  
0       Spotify  
1       YouTube  
2   Apple Music  
3       YouTube  
4      Zing MP3  
..          ...  
95      YouTube  
96      Spotify

In [ ]:
import os
from google.colab import userdata

api_token = userdata.get("api_token")
os.environ["REPLICATE_API_TOKEN"] = api_token

In [ ]:
from langchain_community.llms import Replicate

parameters = {
  "top_k": 5,
  "top_p": 1.0,
  "max_tokens": 100,
  "min_tokens": 0,
  "random_seed": None,
  "repetition_penalty": 1.0,
  "temperature": 0.7,
  "stopping_criteria": "length (256 tokens)",
  "stopping_sequence": None
}

llm = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    input=parameters,
)

In [ ]:
def classify_music_by_year(df_sorted, year_column, title_column, artist_column, platform_column):
    # Mendapatkan semua tahun yang ada di dataset
    years = df_sorted[year_column].unique()

    # Menyiapkan prompt untuk model AI berdasarkan data yang ada, untuk setiap tahun
    top_songs_result = {}  # Mendefinisikan variabel untuk menyimpan hasil klasifikasi
    top_artists_result = {}  # Mendefinisikan variabel untuk menyimpan hasil klasifikasi
    top_platforms_result = {}  # Mendefinisikan variabel untuk menyimpan hasil klasifikasi

    for year in years:
        # Menyaring data berdasarkan tahun
        year_data = df_sorted[df_sorted[year_column] == year]

        # Mempersiapkan prompt untuk setiap tahun
        top_songs_prompt = f"Classify the top 5 most listened songs for year {year} from the dataset: {year_data[[year_column, title_column]].to_dict(orient='records')}"
        top_artists_prompt = f"Classify the top 5 most popular artists for year {year} from the dataset: {year_data[[year_column, artist_column]].to_dict(orient='records')}"
        top_platforms_prompt = f"Classify the top 5 most used platforms for year {year} from the dataset: {year_data[[year_column, platform_column]].to_dict(orient='records')}"

        # Menjalankan model AI untuk mendapatkan hasil
        top_songs_result[year] = llm.invoke(top_songs_prompt)
        top_artists_result[year] = llm.invoke(top_artists_prompt)
        top_platforms_result[year] = llm.invoke(top_platforms_prompt)

    return top_songs_result, top_artists_result, top_platforms_result

In [ ]:
top_songs, top_artists, top_platforms = classify_music_by_year(df_sorted, 'Release_Year', 'Song_Title', 'Artist', 'Platform')

for year in top_songs:
    print(f"\nTop 5 Most Listened Songs in {year} (from AI Model):")
    print(top_songs[year])

    print(f"\nTop 5 Most Popular Artists in {year} (from AI Model):")
    print(top_artists[year])

    print(f"\nTop 5 Most Used Platforms in {year} (from AI Model):")
    print(top_platforms[year])


Top 5 Most Listened Songs in 2010 (from AI Model):
To classify the top 5 most listened songs for the year 2010 from the given dataset, we would need additional information such as listening statistics (e.g., the number of streams or sales) for each song. The dataset provided only contains song titles and their release year. 

However, since the task is to list the songs based on the year 2010, I can simply sort the songs by their release year and list the top 5 according to

Top 5 Most Popular Artists in 2010 (from AI Model):
To classify the top 5 most popular artists for the year 2010 from the given dataset, we need to count the occurrences of each artist. However, there seems to be a discrepancy in the dataset as some entries are collaborations involving artists from different years, which complicates direct ranking based on a single year's popularity. For the purpose of this classification, I'll assume we consider only solo or primary artist entries from 2010, dis

Top 5 Most Used 

# Percobaan

In [ ]:
output = llm.invoke("what is your name?")
print(output)

I don't have a personal name, but you can call me Assistant. I'm here to help answer your questions to the best of my ability. How can I assist you today?


In [ ]:
list_1 = (7, 8, 2, 5, 2, 11, 8)

In [ ]:
list_1[:4]

(7, 8, 2, 5)

In [ ]:
for ember in list_1:
  print("Isi")
  print(ember)
  print()

Isi
7

Isi
8

Isi
2

Isi
5

Isi
2

Isi
11

Isi
8



In [ ]:
data = [
    "With millions of fans watching his every move as IShowSpeed, Darren Watkins Jr. has made a name for himself as a streamer. At just 19 years old, his online presence is growing rapidly.",
    "Known for his lightning-fast footwork on the field, Antony Mateus plays professionally as a football player. He's 25, and already considered one of the rising stars in his league.",
    "The name Barack Obama is etched in history. Now 63, the former president continues to inspire through his speeches and books.",
    "After years behind bars, Lucia is back on the streets of Vice City at 29. She is ready to reclaim her life, navigating crime and chaos as a fierce criminal in the heart of GTA 6.",
    "Selling out stadiums across the world, Taylor Swift is far more than just a 34-year-old singer. Her influence spans music, fashion, and culture."
]

In [ ]:
{
    "name" : "Darren Watkins Jr.",
    "age" : 19,
    "job" : "Streamer"
}

{'name': 'Darren Watkins Jr.', 'age': 19, 'job': 'Streamer'}

In [ ]:
for row in data:
    print(f"Review: {row}")
    prompt = f"""Extract the person's name, age, and job from the following text and output it in JSON format with keys: name, age, job.
Text: {row}
JSON:"""
    output = llm.invoke(prompt)
    print(f"AI OUTPUT: {output}")
    print("=======================")

Review: With millions of fans watching his every move as IShowSpeed, Darren Watkins Jr. has made a name for himself as a streamer. At just 19 years old, his online presence is growing rapidly.
AI OUTPUT: {
  "name": "Darren Watkins Jr.",
  "age": 19,
  "job": "streamer"
}
Review: Known for his lightning-fast footwork on the field, Antony Mateus plays professionally as a football player. He's 25, and already considered one of the rising stars in his league.
AI OUTPUT: {
  "name": "Antony Mateus",
  "age": 25,
  "job": "football player"
}
Review: The name Barack Obama is etched in history. Now 63, the former president continues to inspire through his speeches and books.
AI OUTPUT: {
  "name": "Barack Obama",
  "age": 63,
  "job": "former president, speaker, author"
}
Review: After years behind bars, Lucia is back on the streets of Vice City at 29. She is ready to reclaim her life, navigating crime and chaos as a fierce criminal in the heart of GTA 6.
AI OUTPUT: {
  "name": "Lucia",
  "ag

In [ ]:
!pip install pandas
!pip install -U langchain langchain-community langchain-experimental replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 7.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df = pd.read_csv("/content/favorite_music_dataset.csv")
df

,Song_Title,Artist,Genre,Release_Year,Duration_Minutes,Listened_Date,Platform
0,Faded,Alan Walker,Electronic,2024,4.02,2024-02-10,Apple Music
1,Blinding Lights,The Weeknd,Synth-pop,2018,4.45,2024-05-30,Zing MP3
2,Faded,Alan Walker,Electronic,2024,4.86,2024-05-07,Zing MP3
3,Faded,Alan Walker,Electronic,2012,4.92,2024-03-19,YouTube
4,Bohemian Rhapsody,Queen,Rock,2023,3.90,2024-01-16,Apple Music
...,...,...,...,...,...,...,...
95,Counting Stars,OneRepublic,Pop Rock,2013,3.40,2024-05-30,Spotify
96,Senorita,Shawn Mendes & Camila Cabello,Latin Pop,2024,2.75,2024-06-11,Apple Music
97,Counting Stars,OneRepublic,Pop Rock,2010,3.57,2024-01-03,Zing MP3
98,Blinding Lights,The Weeknd,Synth-pop,2012,4.04,2024-06-13,Apple Music
